In [4]:
import scipy.io
from pathlib import Path
import numpy as np
import os 

In [ ]:
def combine_sm_fm(subj_id):
    save_path = Path(f"/home/chanae/thesis/combined_matrices/{subj_id}.mat") 
    
    # if save_path.exists():
    #     return

    sm_path = Path("/home/chanae/thesis/structural_matrices") / subj_id / "structural_matrix_Yeo_dmn_fpn.npy"
    fm_path = Path("/home/chanae/thesis/functional_matrices") / subj_id / "functional_matrix_Yeo.npy"

    if sm_path.exists() and fm_path.exists():
        struc_matrix = np.load(sm_path)
        func_matrix = np.load(fm_path)
        comb_matrix = {"sc": struc_matrix, "fc":func_matrix}

        scipy.io.savemat(save_path, comb_matrix)

# test_matrices = ['100307', '102109', '103010', '103515', '104416', '105014', '105620', '108020', '108323', '108525']

# for m in test_matrices:
#     combine_sm_fm(m)

subj_list = os.listdir("/home/lars/Data__lars/HCP_1200/")
print(len(subj_list))

for subj in subj_list:
    combine_sm_fm(subj)


In [ ]:
def aggregate_upper_triangles(data_path):

    # Get the list of .mat files for all subjects
    subj_list = os.listdir(data_path)
    subj_list.sort()

    # Get the num of combined fc+sc .mat files in the directory (should be 1052)
    num_files = len([entry for entry in subj_list if os.path.isfile(os.path.join(data_path, entry))])
    columns = 1275 #(upper-triangle of fpn/dmn matrix incl. diagonal calculated by (n*(n+1))/2)

    # Initialize a dictionary to store the flattened upper triangles
    flattened_triangles = {'sc': np.zeros((num_files, columns)), 'fc': np.zeros((num_files, columns))}  

    # Iterate over all .mat files in the specified directory
    for i, subj in enumerate(subj_list):
        # Load the .mat file
        mat = scipy.io.loadmat(os.path.join(data_path, subj))
        
        # For each subj, the matrices are stored in variables 'sc' and 'fc'
        for matrix_name in ['sc', 'fc']:                
            # Extract the upper triangle of the matrix                      
            upper_triangle = mat[matrix_name][np.triu_indices(50)]
            assert upper_triangle.ndim == 1
            assert len(upper_triangle) == columns

            # Add the upper triangle to the dictionary
            flattened_triangles[matrix_name][i] = upper_triangle

        
    # Specify how many subjects will be used for training and testing
    train_size = 820 # ensure the size is a multiple of batch_size (10) - 78%
    test_size = num_files - train_size

    # Separate into train and test dictionaries
    train_dict = {'sc': flattened_triangles['sc'][:train_size], 'fc': flattened_triangles['fc'][:train_size]}
    test_dict = {'sc': flattened_triangles['sc'][train_size:], 'fc': flattened_triangles['fc'][train_size:]}    

    # Save the train and test dictionaries as .mat files
    save_path = Path('/home/chanae/thesis/train_test_matrices')
    scipy.io.savemat(save_path/'train.mat', train_dict)
    scipy.io.savemat(save_path/'test.mat', test_dict)

    return 

data_path = '/home/chanae/thesis/combined_matrices'
aggregate_upper_triangles(data_path)

In [ ]:
import matplotlib.pyplot as plt

input_data = scipy.io.loadmat("/home/chanae/thesis/combined_matrices/100307.mat")['sc']
output_data = scipy.io.loadmat("/home/chanae/thesis/combined_matrices/100307.mat")['fc']

plt.figure(figsize=(5,5))
plt.imshow(output_data)

In [ ]:
data = scipy.io.loadmat('/home/chanae/scripts/mapping_SC_FC-master/example_data.mat')
# print(data)
# print(data.keys())
print(data['fc'].shape)

empty_val = np.zeros((67, 67))
empty_val[np.triu_indices(67)] = data['fc'][0]
print(empty_val)

plt.figure(figsize=(5,5))
plt.imshow(empty_val)



In [ ]:
import matplotlib.pyplot as plt

input_data = scipy.io.loadmat("/home/chanae/thesis/combined_matrices/100307.mat")['sc']
output_data = scipy.io.loadmat("/home/chanae/thesis/combined_matrices/100307.mat")['fc']

plt.figure(figsize=(5,5))
plt.imshow(output_data)